In [1]:
import pandas as pd
import numpy as np
import os
import math
import ast
import warnings
warnings.filterwarnings("ignore")

def get_price_volume(row):
    best_bv = row.atb_ladder['v'][0] if row.atb_ladder else 0
    best_lv = row.atl_ladder['v'][0] if row.atl_ladder else 0
    return pd.Series([best_bv, best_lv])

def get_EP(row):
    best_BV = row.best_BV
    best_LV = row.best_LV
    best_BP = row.back_best
    best_LP = row.lay_best
    P_up = best_BV/(best_BV+best_LV)
    xprice = (best_BP *(1 - P_up)) + (best_LP * P_up)
    return pd.Series([xprice])

def get_spread(row):
    amt_on_market = row.traded_volume
    total_traded = row.total_volume
    traded_prices = row.traded_volume_ladder['p']
    actual_spread = row.best_LV - row.best_BV
    tick = get_tick((row.back_best+row.lay_best)/2)
    xprice = row.expected_price
    liquidity_ratio = amt_on_market/total_traded * 100
    stdev = np.std(traded_prices)
    spread = max(1, round(pow(math.e, -liquidity_ratio)*actual_spread/2*(1+stdev)))
    spread = (spread + 1)/2
    # for horses we want to back more
    back_BP = float('%.2f'%(tick_round(xprice + spread * tick))) # back price (7)
    back_LP = float('%.2f'%(tick_round(xprice - ((spread + 1) * tick)))) # lay price (8)
    # for horses we want to lay more
    lay_BP = float('%.2f'%(tick_round(xprice + ((spread + 1) * tick)))) # back price (9)
    lay_LP = float('%.2f'%(tick_round(xprice - (spread * tick)))) # lay price (10)
    
    return pd.Series([back_BP, back_LP, lay_BP, lay_LP])
# Function to calculate total liability on the k-th horse

def get_liability(row, tradebook):
    selection = row.selection_id
    selection = tradebook.loc[tradebook['selection_id'] == selection].iloc[0]
    tradebook['lay_v_sum'] = tradebook['lay_trades'].apply(sum_v_values)
    sum_X = tradebook['lay_v_sum'].sum() # Amount layed on all horses(X_i)
    tradebook['back_v_sum'] = tradebook['back_trades'].apply(sum_v_values)
    sum_Y = sum_X + tradebook['back_v_sum'].sum() # Amount backed and layed on all horses (Y_i)

    Y_k = selection.back_trades['v']  # Amounts betted on each back-order (Y_{k,l}) on the K'th horse
    BP_k = selection.back_trades['p'] # Back prices (BP_{k,l}) on the K'th horse
    X_k = selection.lay_trades['v']  # Amounts betted on each lay-order (X_{k,j}) on the K'th horse
    LP_k = selection.lay_trades['p']  # Lay prices (LP_{k,j}) on the K'th horse

    # Indicator function for the k-th horse win
    # I_k = 1/row.expected_price
    I_k = 1

    #changing ik gives interesting results, keeping it at 1 for now to assume if all horses will win we need to match their liability
    # Total liability calculation
    TL_k = sum_X - sum_Y + I_k * (sum(np.multiply(Y_k, BP_k)) - sum(np.multiply(X_k, LP_k)))
    #TL_K = sum(volume layed) - sum(volume layed and backed)
    # + W or Loss * (sum(if kth horse wins, return from lay and back) - sum(amount payable if kth horse wins))   
    '''
    if horse loses then liablity is how much is backed -b
    else if horse wins then liablity is backed -b +profit back -loss lay
    '''
    # print(TL_k)
    return pd.Series([TL_k])

def sum_p_values(row):
    return sum(row['p'])

def sum_v_values(row):
    return sum(row['v'])

def tick_round(price):
    tick = get_tick(price)
    return tick * round(price/tick)

def get_tick(price):
    if price <= 2:
        return 0.01
    elif price <= 3:
        return 0.02
    elif price <=4:
        return 0.05
    elif price <= 6:
        return 0.1
    elif price <= 10:
        return 0.2
    elif price <= 20:
        return 0.5
    elif price <=30:
        return 1
    elif price <= 50:
        return 2
    elif price <= 100:
        return 5
    else:
        return 10
    
def init_tradebook(race_df):
    
    # Initialize the tradebook DataFrame
    tradebook = race_df[['market_id', 'selection_id', 'selection_name', 'venue', 'win']].drop_duplicates()
    tradebook['back_orders'] = None
    tradebook['lay_orders'] = None
    tradebook['back_trades'] = None
    tradebook['lay_trades'] = None

    for col in ['back_orders', 'lay_orders', 'back_trades', 'lay_trades']:
        tradebook[col] = tradebook[col].apply(lambda x: {'p': [], 'v': []})

    return tradebook

def bet_apply_commission(df, com = 0.05):

    # Total Market GPL
    df['market_gpl'] = df.groupby('market_id')['gpl'].transform(sum)

    # Apply 5% commission
    df['market_commission'] = np.where(df['market_gpl'] <= 0, 0, 0.05 * df['market_gpl'])

    # Sum of Market Winning Bets
    df['floored_gpl'] = np.where(df['gpl'] <= 0, 0, df['gpl'])
    df['market_netwinnings'] = df.groupby('market_id')['floored_gpl'].transform(sum)

    # Partition Commission According to Selection GPL
    df['commission'] = np.where(df['market_netwinnings'] == 0, 0, (df['market_commission'] * df['floored_gpl']) / (df['market_netwinnings']))

    # Calculate Selection NPL
    df['npl'] = df['gpl'] - df['commission']

    # Drop excess columns
    df = df.drop(columns = ['floored_gpl', 'market_netwinnings', 'market_commission', 'market_gpl'])

    return(df)

In [6]:
def stream(input, tradebook):
    # print(input.atb_ladder)

    input['atb_ladder'] = [ast.literal_eval(x) for x in input['atb_ladder']]
    input['atl_ladder'] = [ast.literal_eval(x) for x in input['atl_ladder']]
    input['traded_volume_ladder'] = [ast.literal_eval(x) for x in input['traded_volume_ladder']]

    input[['best_BV', 'best_LV']] = input.apply(lambda row: get_price_volume(row), axis=1, result_type='expand')
    input[['expected_price']] = input.apply(lambda row: get_EP(row), axis=1, result_type='expand')
    input[['total_volume']] = input['traded_volume'].sum()
    input[['lay_BP', 'lay_LP', 'back_BP', 'back_LP']] = input.apply(lambda row: get_spread(row), axis=1, result_type='expand')
    
    input['favorites'] = input['bsp'].rank().astype(int)
    # print(input)
    # print(np.sum(1/input['expected_price'].to_numpy()))
    capital = 1
    input.apply(lambda row: trade(row, tradebook, capital), axis=1)

    # limit:= multiplying the overround with the wagered amount

def trade(row, tradebook, capital):
    '''trade/row'''
    try:
        n_horses = len(tradebook)
        # do not trade for least favorite horse
        if row.favorites == n_horses:
            # print(f"not trading: {row.selection_id}")
            return
        # skip trade for row with incomplete data
        if (not row.atb_ladder) or (not row.atl_ladder) or (not row.traded_volume_ladder):
            print(f"not trading: {row.selection_id} @ market_id: {row.market_id}")
            return 

        row['liability'] = get_liability(row, tradebook)
        print(row['liability'])
        print(row['best_BV'], row['best_LV'])
        
        selection = row['selection_id']
        selection = tradebook.loc[tradebook['selection_id'] == selection].iloc[0]

        # table 18 model behaviour
        # liability_k = get_liability(_____, tradebook) # not sure 

        # overround = np.sum(1/input['expected_price'].to_numpy()) - 1 # should be the implied probability, not sure to use expected price or bsp?
        # ^ probably use XP, bsp is unknown until the start of the race
        # limit = overround * X # wagered amount, not sure how to get this

        # behaviour = 'backing' if liability_k < 0 else ('laying' if liability_k > limit else 'do not trade')
        
        # ORDER SIZING
        # top 3 favorite horses get 75% of the capital
        if row.favorites <= 3:
            proportion = 0.75 / 3
        else:
            # rest: allocate the remaining 25% with decreasing sequence
            remaining_horses = n_horses - 3
            proportion = (0.25 / remaining_horses) * (n_horses - row.favorites)
        # print(row.favorites, proportion)
        stake = capital * proportion

        if row.liability.values[0] < 0:
            # print('prefer to back')  # Submit the order
            selection.back_orders['p'].append(row.back_BP)
            selection.back_orders['v'].append(stake)
        elif row.liability.values[0] > 1000:
            # if greater than limit lay
            # print('prefer to lay')
            selection.lay_orders['p'].append(row.lay_LP)
            selection.lay_orders['v'].append(stake)
        else:
            # print('trade')
            # otherwise trade both sides with largest spread  # Submit the order
            selection.back_orders['p'].append(row.back_BP)
            selection.back_orders['v'].append(stake)

            selection.lay_orders['p'].append(row.lay_LP)
            selection.lay_orders['v'].append(stake)

        for idx, back in enumerate(selection.back_orders['p']):
            if row.back_best >= back:
                selection.back_trades['p'].append(back)
                selection.back_trades['v'].append(selection.back_orders['v'][idx])
                selection.back_orders['p'].pop(idx)
                selection.back_orders['v'].pop(idx)
                #append to trade book

        for idx, lay in enumerate(selection.lay_orders['p']):
            if row.lay_best <= lay:
                selection.lay_trades['p'].append(lay)
                selection.lay_trades['v'].append(selection.lay_orders['v'][idx])
                selection.lay_orders['p'].pop(idx)
                selection.lay_orders['v'].pop(idx)
    except Exception as e:
            print(e, row)

In [7]:
# initialise trade book
month = '2023_12'
output_dir = f'trade_result/{month}'
# Create output folder
os.makedirs(output_dir, exist_ok=True)

monthly_data = pd.read_csv(f'extracted_data/{month}/{month}_preprocessed.csv')
race_groups = monthly_data.groupby('market_id')

In [8]:
def yield_chunks(df, chunk_size):
    for i in range(0, len(df), chunk_size):
        yield df.iloc[i:i+chunk_size]

monthly_tradebook = pd.DataFrame()

for market_id, race_df in race_groups:
    print(f'Trading for market_id: {market_id}')
    tradebook = init_tradebook(race_df)
    chunksize = len(tradebook['selection_id'].unique().tolist()) # get num runners in market

    # start trading for each tick in the market stream
    for chunk in yield_chunks(race_df, chunksize):
        stream(chunk, tradebook)
        # break
    tradebook = tradebook.drop('back_orders', axis=1)
    tradebook = tradebook.drop('lay_orders', axis=1)
    monthly_tradebook = pd.concat([monthly_tradebook, tradebook], ignore_index=True)
    break

# monthly_tradebook.to_csv(f'{output_dir}/{month}_tradebook.csv', index=False)

Trading for market_id: 1.221906108
0    0
dtype: int64
6.6 1.15
0    0
dtype: int64
3.78 1.09
0    0
dtype: int64
1.47 1.09
0    0
dtype: int64
1.47 1.09
0    0
dtype: int64
1.47 1.09
0    0
dtype: int64
1.47 1.09
0    0
dtype: int64
6.09 3.7
0    0
dtype: int64
0.95 1.09
0    0
dtype: int64
0.07 1.09
0    0
dtype: int64
0.07 1.09
0    0
dtype: int64
4.55 1.09
0    0
dtype: int64
7.86 1.09
0    0
dtype: int64
5.55 1.09
0    0
dtype: int64
10.17 1.09
0    0
dtype: int64
5.55 1.09
0    0
dtype: int64
5.55 1.09
0    0
dtype: int64
2.64 4.0
0    0
dtype: int64
0.21 1.39
0    0
dtype: int64
1.77 2.43
0    0
dtype: int64
4.58 2.76
0    36.5625
dtype: float64
10.14 2.76
0    80.46875
dtype: float64
10.14 2.76
0    99.375
dtype: float64
5.63 4.92
0    111.875
dtype: float64
5.63 4.92
0    118.28125
dtype: float64
5.63 4.92
0    118.28125
dtype: float64
6.38 4.23
0    118.28125
dtype: float64
7.14 4.23
0    118.28125
dtype: float64
7.14 4.23
0    118.28125
dtype: float64
7.14 4.23
0    118.2812

In [79]:
monthly_tradebook

,selection_id,selection_name,win,back_trades,lay_trades,lay_v_sum,back_v_sum
0,64343036,1. Midtown Boss,1,"{'p': [2.06, 2.08, 2.1, 2.1, 2.1, 2.1, 2.1, 2....","{'p': [2.06, 2.08, 2.08, 2.08, 2.08, 2.08, 2.0...",16.200000,109.400000
1,64343037,2. Lulu Darling,0,"{'p': [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50....","{'p': [55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55....",38.600000,12.200000
2,42201546,3. Magnum Bullet,0,"{'p': [18.0, 18.5, 18.5, 18.5, 18.5, 18.5, 18....","{'p': [18.5, 18.5, 18.5, 18.5, 18.5, 18.5, 18....",6.000000,9.400000
3,64343038,4. Mornington Glory,0,"{'p': [4.8, 4.8, 4.9, 4.9, 4.9, 4.8, 4.9, 4.9,...","{'p': [], 'v': []}",0.000000,2.400000
4,38687860,5. Daily Bugle,0,"{'p': [15.0, 15.0, 15.5, 16.0, 16.0, 15.5, 16....","{'p': [19.0, 19.0, 19.0, 19.0, 18.5, 18.5, 18....",32.600000,81.400000
...,...,...,...,...,...,...,...
676,41552565,6. Jupitus,0,"{'p': [42.0, 50.0, 48.0, 60.0, 30.0, 30.0, 30....","{'p': [], 'v': []}",0.000000,11.678571
677,65460475,8. Jucconi,0,"{'p': [17.5, 17.0, 16.0, 15.0, 15.5, 19.0, 20....","{'p': [], 'v': []}",0.000000,5.285714
678,65460476,10. Big Watch,0,"{'p': [3.95, 3.8, 3.55, 3.65, 3.65, 3.6, 3.7, ...","{'p': [], 'v': []}",0.000000,70.428571
679,51576918,11. Terrestar,0,"{'p': [24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24....","{'p': [], 'v': []}",0.000000,91.178571


In [80]:
def calculate_back_lay_profit_liability(df):
    '''
    if horse loses then liablity is how much is backed -b + lay profit
    else if horse wins then liablity is +back profit -loss lay
    '''

    #changed label from profit to return because we are summing that column

    df['back_liability'] = 0
    df['back_return'] = 0
    df['lay_liability'] = 0
    df['lay_return'] = 0
    
    for index, row in df.iterrows():
        back_prices = np.array(row['back_trades']['p'])
        back_volumes = np.array(row['back_trades']['v'])
        
        lay_prices = np.array(row['lay_trades']['p'])
        lay_volumes = np.array(row['lay_trades']['v'])
        
        back_liability = sum(back_volumes)
        if row['win'] == 1:
            back_return = np.dot(back_prices, back_volumes) - sum(back_volumes)
            #profit does not include outlay, ie. $1@2.00 = $2 return - $1 stake => $1 profit
        else:
            back_return = -back_liability # lose your backing stake
        
        lay_return = sum(lay_volumes)
        if row['win'] == 1:
            lay_liability = np.dot(lay_volumes, (lay_prices - 1))
            lay_return = -lay_liability
        else:
            lay_liability = 0 # if horse loses then no liablity
        
        df.at[index, 'back_liability'] = back_liability
        df.at[index, 'back_return'] = back_return
        df.at[index, 'lay_liability'] = lay_liability
        df.at[index, 'lay_return'] = lay_return
    
    return df
# print(tradebook)
summary = calculate_back_lay_profit_liability(monthly_tradebook)
summary

,selection_id,selection_name,win,back_trades,lay_trades,lay_v_sum,back_v_sum,back_liability,back_return,lay_liability,lay_return
0,64343036,1. Midtown Boss,1,"{'p': [2.06, 2.08, 2.1, 2.1, 2.1, 2.1, 2.1, 2....","{'p': [2.06, 2.08, 2.08, 2.08, 2.08, 2.08, 2.0...",16.200000,109.400000,109.400000,124.544000,17.2,-17.200000
1,64343037,2. Lulu Darling,0,"{'p': [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50....","{'p': [55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55....",38.600000,12.200000,12.200000,-12.200000,0.0,38.600000
2,42201546,3. Magnum Bullet,0,"{'p': [18.0, 18.5, 18.5, 18.5, 18.5, 18.5, 18....","{'p': [18.5, 18.5, 18.5, 18.5, 18.5, 18.5, 18....",6.000000,9.400000,9.400000,-9.400000,0.0,6.000000
3,64343038,4. Mornington Glory,0,"{'p': [4.8, 4.8, 4.9, 4.9, 4.9, 4.8, 4.9, 4.9,...","{'p': [], 'v': []}",0.000000,2.400000,2.400000,-2.400000,0.0,0.000000
4,38687860,5. Daily Bugle,0,"{'p': [15.0, 15.0, 15.5, 16.0, 16.0, 15.5, 16....","{'p': [19.0, 19.0, 19.0, 19.0, 18.5, 18.5, 18....",32.600000,81.400000,81.400000,-81.400000,0.0,32.600000
...,...,...,...,...,...,...,...,...,...,...,...
676,41552565,6. Jupitus,0,"{'p': [42.0, 50.0, 48.0, 60.0, 30.0, 30.0, 30....","{'p': [], 'v': []}",0.000000,11.678571,11.678571,-11.678571,0.0,0.000000
677,65460475,8. Jucconi,0,"{'p': [17.5, 17.0, 16.0, 15.0, 15.5, 19.0, 20....","{'p': [], 'v': []}",0.000000,5.285714,5.285714,-5.285714,0.0,0.000000
678,65460476,10. Big Watch,0,"{'p': [3.95, 3.8, 3.55, 3.65, 3.65, 3.6, 3.7, ...","{'p': [], 'v': []}",0.000000,70.428571,70.428571,-70.428571,0.0,0.000000
679,51576918,11. Terrestar,0,"{'p': [24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24....","{'p': [], 'v': []}",0.000000,91.178571,91.178571,-91.178571,0.0,0.000000


In [81]:
summary.to_csv(f'{output_dir}/{month}_summary.csv', index=False)

In [82]:
back_return = summary['back_return'].sum()
lay_return = summary['lay_return'].sum()
total_return = back_return + lay_return
total_return

1067.6730757297685

In [83]:
total_trades = (summary['back_v_sum'] + summary['lay_v_sum']).sum()
total_trades

47028.82286741043

In [84]:
total_return/total_trades * 100

2.2702526038975854

In [ ]:
def trade2(df, stake = 1, back_odds = 'bsp', lay_odds = 'bsp'):

    """
    Betting DF should always contain: expected_price, and win (binary encoded), and the specified odds column columns
    """
    df['atb_ladder'] = [ast.literal_eval(x) for x in df['atb_ladder']]
    df['atl_ladder'] = [ast.literal_eval(x) for x in df['atl_ladder']]
    df['traded_volume_ladder'] = [ast.literal_eval(x) for x in df['traded_volume_ladder']]
    df[['best_BV', 'best_LV']] = df.apply(lambda row: get_price_volume(row), axis=1, result_type='expand')
    df[['expected_price']] = df.apply(lambda row: get_EP(row), axis=1, result_type='expand')
    df[['total_volume']] = df['traded_volume'].sum()
    df[['lay_BP', 'lay_LP', 'back_BP', 'back_LP']] = df.apply(lambda row: get_spread(row), axis=1, result_type='expand')
    
    df['bet_side'] = np.where((df["lay_BP"] >= df[back_odds]) & (df["lay_LP"] <= df[lay_odds]),
                            "P", # PUSH
                            np.where(
                                df["lay_BP"] < df[back_odds],
                                "B",
                                "L"
                            )
                       )

    df['stake'] = np.where(df['bet_side'] == "P", # PUSH
                           0,
                           np.where(
                             df['bet_side'] == "B",
                             ( (1 / df['back_BP']) - (1 / df[back_odds]) ) / (1 - (1 / df[back_odds])),
                             ( (1 / df[lay_odds]) - (1 / df['back_LP']) ) / (1 - (1 / df[lay_odds])),
                           )
                          )

    df['gpl'] = np.where(df['bet_side'] == "B", 
                         np.where(df['win'] == 1, df['stake'] * (df[back_odds]-1), -df['stake']), # PL for back bets
                         np.where(df['win'] == 1, -df['stake'] * (df[lay_odds]-1), df['stake']) # PL for lay bets
                        )

    # Apply commission and NPL
    # df = bet_apply_commission(df, com=0)

    return(df)

# Create simple PL and POT table
def bet_eval_metrics(d, side = False):

    if side:
        metrics = (d
         .groupby('bet_side', as_index=False)
         .agg({"npl": "sum", "stake": "sum"})
         .assign(pot=lambda x: x['npl'] / x['stake'])
        )
    else:
        metrics = pd.DataFrame(d
         .agg({"npl": "sum", "stake": "sum"})
        ).transpose().assign(pot=lambda x: x['npl'] / x['stake'])

    return(metrics[metrics['stake'] != 0])

# Cumulative PL by market to visually see trend and consistency
def bet_eval_chart_cPl(d):

    d = (
        d
        .groupby('market_id')
        .agg({'npl': 'sum'})
    )

    d['market_number'] = np.arange(len(d))
    d['cNpl'] = d.npl.cumsum()

    chart = px.line(d, x="market_number", y="cNpl", title='Cumulative Net Profit', template='simple_white')

    return(chart)

# Initialize an empty DataFrame to hold the results
bets = pd.DataFrame()

# Iterate over each group (each race)
for chunk in pd.read_csv('test_data.csv', chunksize=chunksize):
    # print(len(race_df['selection_id'].unique()))
    # Process the race using bet_random
    processed_race = trade2(chunk, stake=1)
    # Append the processed race to the master result DataFrame
    bets = pd.concat([bets, processed_race], ignore_index=True)

bet_eval_metrics(bets, side = True)